In [1]:
import pandas as pd
dictionary=pd.read_csv("Dictionary.csv")
dictionary.head()

,Word,Sequence Number,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Constraining,Superfluous,Interesting,Modal,Irr_Verb,Harvard_IV,Syllables,Source
0,AARDVARK,1,81,5.690000e-09,3.070000e-09,5.780000e-07,45,0,0,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,2,1.400000e-10,8.220000e-12,7.840000e-09,1,0,0,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,8,5.620000e-10,1.690000e-10,7.100000e-08,7,0,0,0,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,5,3.510000e-10,1.730000e-10,7.530000e-08,5,0,0,0,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,1752,1.230000e-07,1.200000e-07,1.110000e-05,465,0,0,0,0,0,0,0,0,0,0,3,12of12inf


In [2]:
Negative=list(dictionary[dictionary.Negative>0]["Word"])
Positive=list(dictionary[dictionary.Positive>0]["Word"])
Uncertainty=list(dictionary[dictionary.Uncertainty>0]["Word"])
Litigious=list(dictionary[dictionary.Litigious>0]["Word"])
Constraining=list(dictionary[dictionary.Constraining>0]["Word"])
Superfluous=list(dictionary[dictionary.Superfluous>0]["Word"])
Interesting=list(dictionary[dictionary.Interesting>0]["Word"])
Modal1=list(dictionary[dictionary.Modal==1]["Word"])
Modal2=list(dictionary[dictionary.Modal==2]["Word"])
Modal3=list(dictionary[dictionary.Modal==3]["Word"])


In [4]:
from elasticsearch import Elasticsearch
es = Elasticsearch(["http://search-banktweets-oe3qkimmg7aavxn2djiwrm5bi4.us-east-1.es.amazonaws.com:80"])
res=es.search(index="banktweets", doc_type="tweets",scroll="1m",body={"from":0, "size":100, "query": {"match": {"text":"Bank of America"}}})
scrollId=res["_scroll_id"]

In [5]:
import re
dataset=[]
scrollres=es.scroll(scroll_id=scrollId,scroll="1m") 
scrollres

{'_scroll_id': 'DnF1ZXJ5VGhlbkZldGNoBQAAAAAABU4iFjNjbGhIOU1QUldDSVpqTm1BTVhfUncAAAAAAAVOIxYzY2xoSDlNUFJXQ0laak5tQU1YX1J3AAAAAAAFTiQWM2NsaEg5TVBSV0NJWmpObUFNWF9SdwAAAAAABU4lFjNjbGhIOU1QUldDSVpqTm1BTVhfUncAAAAAAAVOJhYzY2xoSDlNUFJXQ0laak5tQU1YX1J3',
 '_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'AVuOqfbyQJg5hlQOPTH_',
    '_index': 'banktweets',
    '_score': 7.325703,
    '_source': {'text': '#noice @ Bank of America https://t.co/BNldAwTcqu',
     'time': 1492747155000,
     'user': 'Charles Hammett'},
    '_type': 'tweets'},
   {'_id': 'AVuQ01o5QJg5hlQOPUKP',
    '_index': 'banktweets',
    '_score': 7.325703,
    '_source': {'text': '@jenniespaghetty three words. bank... of.... america. -_-',
     'time': 1492783421000,
     'user': 'Daniel. 🔞'},
    '_type': 'tweets'},
   {'_id': 'AVuRKy2yQJg5hlQOPUfQ',
    '_index': 'banktweets',
    '_score': 7.325703,
    '_source': {'text': 'bouta step to this Bank of America ..',
     'time': 1492789177000,
  

In [6]:
while( len(scrollres["hits"]["hits"])>0):    
    for hit in scrollres['hits']['hits']:
        neg,pos,unc,lit,con,sup,intr,mod1,mod2,mod3=0,0,0,0,0,0,0,0,0,0
        p = re.compile("[^a-zA-Z]?")
        stringArray=p.split(hit["_source"]["text"])
        for word in stringArray:
            WORD=word.upper()
            if WORD in Negative:
                neg=neg+1
            if WORD in Positive:
                pos=pos+1
            if WORD in Uncertainty:
                unc=unc+1
            if WORD in Litigious:
                lit=lit+1
            if WORD in Constraining:
                con=con+1
            if WORD in Superfluous:
                sup=sup+1
            if WORD in Interesting:
                intr=intr+1
            if WORD in Modal1:
                mod1=mod1+1
            if WORD in Modal2:
                mod2=mod2+1
            if WORD in Modal3:
                mod3=mod3+1
        array=[hit["_source"]["time"],neg,pos,unc,lit,con,sup,intr,mod1,mod2,mod3]
        dataset.append(array)
    scrollres=es.scroll(scroll_id=scrollId,scroll="1m")


E:\Study\Python\Anaconda03\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: split() requires a non-empty pattern match.


In [7]:
df=pd.DataFrame(dataset,columns=['timestamp','neg', 'pos', 'unc', 'lit', 'con','sup','intr','Mod1','Mod2','Mod3'])
df.head()

,timestamp,neg,pos,unc,lit,con,sup,intr,Mod1,Mod2,Mod3
0,1492747155000,0,0,0,0,0,0,0,0,0,0
1,1492783421000,0,0,0,0,0,0,0,0,0,0
2,1492789177000,0,0,0,0,0,0,0,0,0,0
3,1492793080000,0,1,0,0,0,0,0,0,0,0
4,1492797382000,1,0,0,0,0,0,0,0,0,0


In [8]:
from datetime import datetime
def timetransfer(ts):
    return datetime.fromtimestamp(ts/1000+600).strftime('%Y-%m-%d %H:%M:%S')
df["time"]=df["timestamp"].map(lambda t:timetransfer(t))
df.head()


,timestamp,neg,pos,unc,lit,con,sup,intr,Mod1,Mod2,Mod3,time
0,1492747155000,0,0,0,0,0,0,0,0,0,0,2017-04-21 00:09:15
1,1492783421000,0,0,0,0,0,0,0,0,0,0,2017-04-21 10:13:41
2,1492789177000,0,0,0,0,0,0,0,0,0,0,2017-04-21 11:49:37
3,1492793080000,0,1,0,0,0,0,0,0,0,0,2017-04-21 12:54:40
4,1492797382000,1,0,0,0,0,0,0,0,0,0,2017-04-21 14:06:22


In [9]:
df=df.set_index(pd.DatetimeIndex(df['time'])).sort_index().drop("timestamp",axis=1)
df["count"]=1
df=df.resample("5T").sum()
df.head()


,neg,pos,unc,lit,con,sup,intr,Mod1,Mod2,Mod3,count
time,,,,,,,,,,,
2017-04-16 18:25:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2017-04-16 18:30:00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2017-04-16 18:35:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2017-04-16 18:40:00,2.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,12.0
2017-04-16 18:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0


In [10]:
import pandas as pd
import numpy as np
import urllib
import datetime as dt
import matplotlib.pyplot as plt
 
def get_google_data(symbol, period, window):
    url_root = 'http://www.google.com/finance/getprices?i='
    url_root += str(period) + '&p=' + str(window)
    url_root += 'd&f=d,o,h,l,c,v&df=cpct&q=' + symbol
    response = urllib.request.urlopen(url_root)
    data = str(response.read()).split('\\n')
#actual data starts at index = 7
#first line contains full timestamp,
#every other line is offset of period from timestamp
    parsed_data = []
    anchor_stamp = ''
    end = len(data)
    for i in range(7, end):
        cdata = data[i].split(',')
        if 'a' in cdata[0]:
#first one record anchor timestamp
            anchor_stamp = cdata[0].replace('a', '')
            cts = int(anchor_stamp)
        else:
            try:
                coffset = int(cdata[0])
                cts = int(anchor_stamp) + (coffset * period)
                parsed_data.append((dt.datetime.fromtimestamp(float(cts)), float(cdata[1]), float(cdata[2]), float(cdata[3]), float(cdata[4]), float(cdata[5])))
            except:
                pass # for time zone offsets thrown into data
    df = pd.DataFrame(parsed_data)
    df.columns = ['ts', 'o', 'h', 'l', 'c', 'v']
    df.index = df.ts
    del df['ts']
    return df

In [11]:
BOA = get_google_data('BAC', 300, 14)
BOA.head()

,o,h,l,c,v
ts,,,,,
2017-04-06 09:35:00,23.080,23.19,23.02,23.14,4209683.0
2017-04-06 09:40:00,23.130,23.14,23.06,23.08,2496436.0
2017-04-06 09:45:00,23.150,23.19,23.10,23.13,3018909.0
2017-04-06 09:50:00,23.025,23.15,22.96,23.15,3508159.0
2017-04-06 09:55:00,23.170,23.18,23.02,23.02,1954324.0


In [12]:
BOA["C-O"]=BOA["c"]-BOA["o"]
def binary(v):
    if(v>0):
        return 1
    else:
        return 0
BOA["Target"]=BOA["C-O"].map(lambda t:binary(t))  
BOA.head()

,o,h,l,c,v,C-O,Target
ts,,,,,,,
2017-04-06 09:35:00,23.080,23.19,23.02,23.14,4209683.0,0.060,1
2017-04-06 09:40:00,23.130,23.14,23.06,23.08,2496436.0,-0.050,0
2017-04-06 09:45:00,23.150,23.19,23.10,23.13,3018909.0,-0.020,0
2017-04-06 09:50:00,23.025,23.15,22.96,23.15,3508159.0,0.125,1
2017-04-06 09:55:00,23.170,23.18,23.02,23.02,1954324.0,-0.150,0


In [13]:
BOAData=BOA.join(df,how="left").dropna().drop(["o","h","l","c","v","C-O"],axis=1)
BOAData

,Target,neg,pos,unc,lit,con,sup,intr,Mod1,Mod2,Mod3,count
ts,,,,,,,,,,,,
2017-04-17 09:40:00,1,3.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,9.0
2017-04-17 09:45:00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
2017-04-17 09:50:00,0,6.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0
2017-04-17 09:55:00,0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0
2017-04-17 10:00:00,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
2017-04-17 10:05:00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0
2017-04-17 10:10:00,1,0.0,2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,12.0
2017-04-17 10:15:00,0,4.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,12.0
2017-04-17 10:20:00,0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
